In [1]:
import cv2
import torch
import numpy as np


model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

def detect_objects(image):
    results = model(image) 
    detections = results.xyxy[0] 
    boxes = detections[:, :4].cpu().numpy() 
    confidences = detections[:, 4].cpu().numpy()
    class_ids = detections[:, 5].cpu().numpy().astype(int) 
    indexes = cv2.dnn.NMSBoxes(boxes.tolist(), confidences.tolist(), 0.5, 0.4) 
    return boxes, confidences, class_ids, indexes

cap = cv2.VideoCapture(1) 


if not cap.isOpened():
    print("Error: Could not open webcam")
else:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame")
            break

        boxes, confidences, class_ids, indexes = detect_objects(frame)
        font = cv2.FONT_HERSHEY_PLAIN
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                
                if not np.isnan(x) and not np.isnan(y) and not np.isnan(w) and not np.isnan(h):
                    x, y, w, h = int(x), int(y), int(w), int(h)  
                    
                    label = model.names[class_ids[i]]
                    color = (0, 255, 0) 
                    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                    cv2.putText(frame, label, (x, y - 10), font, 1, color, 2)
                else:
                    print("Warning: Invalid bounding box coordinates")

        cv2.imshow("Webcam", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Working

Using cache found in C:\Users\Hare Krishna/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-5-28 Python-3.11.4 torch-2.3.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
